In [1]:
!pip install fastapi uvicorn nest_asyncio jinja2
!pip install git+https://github.com/openai/whisper.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.4 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-zk56gb79
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-zk56gb79
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
!pip install pyngrok
!pip install python-multipart

khởi tạo api

In [3]:
import tempfile
import whisper
import nest_asyncio
from pyngrok import ngrok, conf
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import uvicorn
from typing import List
import logging
from datetime import datetime

# Thiết lập logging với format chi tiết
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Khởi tạo các thành phần cần thiết
nest_asyncio.apply()

# Cấu hình Ngrok với token
NGROK_AUTH_TOKEN = "2sWjZTSTemDeBnksDgYJvraqVku_7aYAKKMyNoNhKzoUxhHFU"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Khởi tạo FastAPI app
app = FastAPI(
    title="Speech Recognition API",
    description="API for transcribing speech to text using Whisper model",
    version="1.0.0"
)

# Cấu hình CORS - cho phép cross-origin requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["POST", "GET"],
    allow_headers=["*"],
)

# Khởi tạo model Whisper
try:
    model = whisper.load_model("base")
    logger.info("Whisper model loaded successfully")
except Exception as e:
    logger.error(f"Error loading Whisper model: {str(e)}")
    raise

def clean_text(text: str) -> str:
    """
    Hàm làm sạch văn bản từ transcription

    Args:
        text (str): Văn bản cần làm sạch

    Returns:
        str: Văn bản đã được làm sạch
    """
    # Xóa các ký tự đặc biệt
    text = text.replace('"""', '')
    text = text.replace('\n', ' ')
    text = text.replace(')', '')

    # Loại bỏ khoảng trắng thừa
    text = ' '.join(text.split())

    return text

def extract_part1(text: str, conversation: List[str]) -> List[str]:
    """
    Xử lý và trích xuất nội dung Part 1 của bài nói

    Args:
        text (str): Văn bản đầu vào
        conversation (List[str]): List chứa các dòng hội thoại

    Returns:
        List[str]: List cập nhật với nội dung Part 1
    """
    # Đánh dấu phần bắt đầu của Part 1
    conversation.append("--- Part 1 ---")

    # Tìm vị trí bắt đầu và kết thúc của Part 1
    intro_mark = text.find("Now, in this first part")
    ending_part1_mark = text.find("That is the end of part 1")

    if intro_mark != -1 and ending_part1_mark != -1:
        part1_text = text[intro_mark:ending_part1_mark]

        # Tách câu dựa trên dấu chấm và dấu hỏi
        sentences = []
        current_sentence = ""

        for char in part1_text:
            current_sentence += char
            if char in ['?', '.']:
                if current_sentence.strip():
                    sentences.append(current_sentence.strip())
                current_sentence = ""

        # Xử lý từng câu trong Part 1
        i = 0
        while i < len(sentences):
            sentence = sentences[i].strip()

            # Nếu là câu hỏi
            if '?' in sentence:
                # Thêm câu hỏi vào conversation
                conversation.append(f"examiner: {sentence}")

                # Tìm câu trả lời (các câu tiếp theo không phải là câu hỏi)
                answer_parts = []
                j = i + 1
                while j < len(sentences) and '?' not in sentences[j]:
                    answer = sentences[j].strip()
                    if answer:
                        answer_parts.append(answer)
                    j += 1

                # Nếu có câu trả lời, thêm vào conversation
                if answer_parts:
                    conversation.append(f"candidate: {' '.join(answer_parts)}")
                i = j - 1
            i += 1

    return conversation

def extract_part2(text: str, conversation: List[str]) -> List[str]:
    """
    Xử lý và trích xuất nội dung Part 2 của bài nói

    Args:
        text (str): Văn bản đầu vào
        conversation (List[str]): List chứa các dòng hội thoại

    Returns:
        List[str]: List cập nhật với nội dung Part 2
    """
    # Đánh dấu phần bắt đầu của Part 2
    conversation.append("--- Part 2 ---")

    # Tìm vị trí bắt đầu và kết thúc của Part 2
    part2_start = text.find("Now, let's move on to part 2")
    part2_end = text.find("That is the end of part 2")

    if part2_start != -1 and part2_end != -1:
        part2_text = text[part2_start:part2_end]

        # Tìm câu hỏi chính của Part 2
        question_start = part2_text.find("I'd like you to")
        if question_start != -1:
            main_question = part2_text[question_start:].split(".")[0] + "."
            conversation.append(f"examiner: {main_question}")

        # Tìm phần trả lời của thí sinh
        speaking_start = part2_text.find("Start speaking now")
        if speaking_start != -1:
            answer = part2_text[speaking_start:].strip()
            if answer:
                conversation.append(f"candidate: {answer}")

    return conversation

def extract_part3(text: str, conversation: List[str]) -> List[str]:
    """
    Xử lý và trích xuất nội dung Part 3 của bài nói

    Args:
        text (str): Văn bản đầu vào
        conversation (List[str]): List chứa các dòng hội thoại

    Returns:
        List[str]: List cập nhật với nội dung Part 3
    """
    # Đánh dấu phần bắt đầu của Part 3
    conversation.append("--- Part 3 ---")

    # Tìm vị trí bắt đầu của Part 3
    part3_start = text.find("Now let's move on to part 3")
    if part3_start != -1:
        part3_text = text[part3_start:]

        # Tách câu
        sentences = []
        current_sentence = ""

        for char in part3_text:
            current_sentence += char
            if char in ['?', '.']:
                if current_sentence.strip():
                    sentences.append(current_sentence.strip())
                current_sentence = ""

        # Xử lý từng câu trong Part 3
        i = 0
        while i < len(sentences):
            sentence = sentences[i].strip()

            # Nếu là câu hỏi
            if '?' in sentence:
                # Thêm câu hỏi vào conversation
                conversation.append(f"examiner: {sentence}")

                # Tìm câu trả lời
                answer_parts = []
                j = i + 1
                while j < len(sentences) and '?' not in sentences[j]:
                    answer = sentences[j].strip()
                    if answer:
                        answer_parts.append(answer)
                    j += 1

                # Nếu có câu trả lời, thêm vào conversation
                if answer_parts:
                    conversation.append(f"candidate: {' '.join(answer_parts)}")
                i = j - 1
            i += 1

    return conversation

def process_transcription(text: str) -> List[str]:
    """
    Xử lý toàn bộ văn bản transcription thành dạng hội thoại

    Args:
        text (str): Văn bản transcription

    Returns:
        List[str]: Danh sách các dòng hội thoại đã được xử lý
    """
    try:
        # Khởi tạo list chứa các dòng hội thoại
        conversation = []

        # Làm sạch text
        cleaned_text = clean_text(text)

        # Xử lý từng phần
        conversation = extract_part1(cleaned_text, conversation)
        conversation = extract_part2(cleaned_text, conversation)
        conversation = extract_part3(cleaned_text, conversation)

        return conversation

    except Exception as e:
        logger.error(f"Error in process_transcription: {str(e)}")
        return ["Error processing transcription"]

@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    """
    Endpoint xử lý file âm thanh và trả về kết quả hội thoại

    Args:
        file (UploadFile): File âm thanh WAV cần xử lý

    Returns:
        JSONResponse: Kết quả hội thoại dưới dạng JSON
    """
    try:
        # Log thông tin request
        logger.info(f"Received file: {file.filename}")
        start_time = datetime.utcnow()

        # Kiểm tra định dạng file
        if not file.filename.lower().endswith(".wav"):
            return JSONResponse(
                status_code=400,
                content={"error": "Invalid file format. Only WAV files are accepted"}
            )

        # Đọc và xử lý file
        audio_bytes = await file.read()

        # Tạo temporary file để xử lý
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp:
            # Ghi dữ liệu vào temporary file
            tmp.write(audio_bytes)
            tmp.flush()

            # Thực hiện transcribe
            result = model.transcribe(tmp.name, fp16=False)
            transcription = result.get("text", "")

            # Xử lý transcription thành conversation
            conversation = process_transcription(transcription)

            # Log thời gian hoàn thành
            end_time = datetime.utcnow()
            processing_time = (end_time - start_time).total_seconds()
            logger.info(f"Processing completed in {processing_time:.2f} seconds")

            # Trả về kết quả
            return JSONResponse(
                content={"conversation": conversation},
                status_code=200
            )

    except Exception as e:
        logger.error(f"Error processing request: {str(e)}")
        return JSONResponse(
            status_code=500,
            content={"error": "Internal server error during processing"}
        )

@app.get("/health")
async def health_check():
    """
    Endpoint kiểm tra trạng thái hoạt động của server

    Returns:
        dict: Thông tin về trạng thái server
    """
    return {
        "status": "healthy",
        "timestamp": datetime.utcnow().isoformat(),
        "model_loaded": model is not None
    }

def start_server():
    """
    Khởi động server với ngrok tunnel

    Returns:
        str: URL public của ngrok tunnel
    """
    try:
        # Khởi tạo ngrok tunnel
        public_url = ngrok.connect(8000)

        # In đậm thông tin URL trong terminal
        print("\n" + "="*50)
        print("\033[1m\033[92m[Server Information]\033[0m")  # Màu xanh lá, in đậm
        print(f"⏰ Server started at: {datetime.utcnow()}")
        print(f"🌐 Local URL: \033[94mhttp://localhost:8000\033[0m")
        print(f"🚀 Public URL (Ngrok): \033[94m{public_url}\033[0m")
        print(f"📚 API Documentation: \033[94m{public_url}/docs\033[0m")
        print(f"💡 Health Check: \033[94m{public_url}/health\033[0m")
        print("="*50 + "\n")

        # Logging cho file log
        logger.info(f"Server started at: {datetime.utcnow()}")
        logger.info(f"Local URL: http://localhost:8000")
        logger.info(f"Public URL: {public_url}")
        logger.info(f"API Documentation: {public_url}/docs")
        logger.info(f"Health Check: {public_url}/health")

        return public_url

    except Exception as e:
        error_msg = f"Failed to start ngrok tunnel: {str(e)}"
        print(f"\n\033[91m[ERROR] {error_msg}\033[0m")  # Màu đỏ cho lỗi
        logger.error(error_msg)
        raise

if __name__ == "__main__":
    try:
        print("\n\033[1m🚀 Starting Speech Recognition API Server...\033[0m\n")  # In đậm

        # Khởi động server với ngrok
        public_url = start_server()

        print("\n\033[1m📡 Starting FastAPI Server...\033[0m\n")  # In đậm

        # Chạy FastAPI server
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=8000,
            log_level="info"
        )
    except Exception as e:
        error_msg = f"Server startup error: {str(e)}"
        print(f"\n\033[91m[ERROR] {error_msg}\033[0m")  # Màu đỏ cho lỗi
        logger.error(error_msg)

100%|███████████████████████████████████████| 139M/139M [00:04<00:00, 32.0MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper


🚀 Starting Speech Recognition API Server...



INFO:     Started server process [218]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)



[Server Information]
⏰ Server started at: 2025-02-05 01:45:23.182327
🌐 Local URL: http://localhost:8000
🚀 Public URL (Ngrok): NgrokTunnel: "https://c29f-35-227-77-71.ngrok-free.app" -> "http://localhost:8000"
📚 API Documentation: NgrokTunnel: "https://c29f-35-227-77-71.ngrok-free.app" -> "http://localhost:8000"/docs
💡 Health Check: NgrokTunnel: "https://c29f-35-227-77-71.ngrok-free.app" -> "http://localhost:8000"/health


📡 Starting FastAPI Server...

INFO:     42.116.198.240:0 - "POST /transcribe HTTP/1.1" 200 OK
INFO:     42.116.198.240:0 - "POST /transcribe HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [218]
